In [32]:
import numpy as np
import scipy as sp
import sklearn
import nltk
import tensorflow as tf
import matplotlib
import gzip
import random
from collections import defaultdict
import sklearn.decomposition

regularization_rate = 0.01
Reduction_Size = 20
batch_size = 200
max_iter = 50000
learning_rate = 0.0001

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [48]:
data = []
for l in readGz('train.json.gz'):
    data.append(l)


In [49]:
userList = []
businessList = []
userDict = defaultdict(int)
businessDict = defaultdict(int)
visitedDict = defaultdict(int)
for d in data:
    u = d['userID']
    b = d['businessID']
    if u not in userList:
        userList.append(u)
    if b not in businessList:
        businessList.append(b)
    visitedDict[(u,b)]+=1

    
for i in range(len(userList)):
    userDict[userList[i]] = i
for i in range(len(businessList)):
    businessDict[businessList[i]] = i

In [50]:
random.shuffle(data)
train_data = data

#validation_data = data[160000:200000]
negative_pair = []
randomCnt = 0
while(randomCnt) < 40000:
    ruIndex = random.randint(0, len(userList)-1)
    rbIndex = random.randint(0, len(businessList)-1)
    ru = userList[ruIndex]
    rb = businessList[rbIndex]
    if visitedDict[(ru, rb)]==0:
        randomCnt +=1
        negative_pair.append((ru,rb))
#Finished creating dataset

ratings = np.array([d['rating'] for d in train_data])
avgRating = np.mean(ratings)
print avgRating


4.2


In [51]:
def Mean(lst):
    if len(lst) != 0:
        return np.mean(lst)
    else:
        return avgRating

In [52]:
businessRat = [[] for i in range(len(businessList))]
for d in train_data:
    businessRat[businessDict[d['businessID']]].append(d['rating'])
businessAvg = [Mean(businessRat[i]) for i in range(len(businessList))]
print businessAvg[:20]

[5.0, 5.0, 4.0, 3.0, 3.0, 5.0, 4.0, 5.0, 4.0, 4.0]


In [53]:
categoryList = []
for d in train_data:
    for c in d['categories']:
        if c not in categoryList:
            categoryList.append(c)
categoryDict = defaultdict(int)
for c in categoryList:
    categoryDict[c] = categoryList.index(c)

businessCategory = [[0 for j in range(len(categoryList))] for i in range(len(businessList))]
userHistory = [[0 for j in range(len(categoryList))] for i in range(len(userList))]
for d in train_data:
    for c in d['categories']:
        userHistory[userDict[d['userID']]][categoryDict[c]]+=1.0
        businessCategory[businessDict[d['businessID']]][categoryDict[c]]=1


In [58]:
def feature(b,u):
    feat = np.array([businessAvg[businessDict[b]]])
    extension = np.array(userHistory[userDict[u]])
    print ("norm = %f"%np.linalg.norm(extension))

    extension = np.divide(extension, np.linalg.norm(extension))
    cat = np.array(businessCategory[businessDict[b]])
    print ("norm = %f"%np.linalg.norm(cat))
    cat = np.divide(cat, np.linalg.norm(cat))
    extension = np.multiply(extension, cat)
    feat = np.concatenate((feat,extension))
    return feat

In [59]:
train_feature = [feature(d['businessID'], d['userID']) for d in train_data]
train_feature.extend([feature(b,u) for u,b in negative_pair[:10]])
#print train_feature
#validation_feature = [feature(d['businessID'], d['userID']) for d in validation_data].extend([feature(b,u) for u,b in negative_pair])
print train_feature

norm = 1.732051
norm = 1.732051
norm = 1.414214
norm = 1.414214
norm = 1.732051
norm = 1.732051
norm = 1.414214
norm = 1.414214
norm = 1.732051
norm = 1.732051
norm = 1.732051
norm = 1.732051
norm = 1.000000
norm = 1.000000
norm = 1.732051
norm = 1.732051
norm = 1.732051
norm = 1.732051
norm = 1.000000
norm = 1.000000
norm = 1.732051
norm = 1.000000
norm = 1.732051
norm = 1.414214
norm = 1.414214
norm = 1.732051
norm = 1.414214
norm = 1.000000
norm = 1.000000
norm = 1.414214
norm = 1.732051
norm = 1.000000
norm = 1.732051
norm = 1.414214
norm = 1.000000
norm = 1.414214
norm = 1.732051
norm = 1.414214
norm = 1.732051
norm = 1.732051
[array([ 4.        ,  0.33333333,  0.33333333,  0.33333333,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ]), array([ 5. ,  0. ,  0. 